In [1]:
import numpy as np
import dagology as dag
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sp
import example_utils as util

In [2]:
# load dataset
filepath = './datasets/hep-th.txt'
edges, num_vertices = util.load_txt_dataset(filepath) 
N = 1000
top_N_edges, top_N_nodes = util.restrict_top_N(edges, N)
print len(top_N_edges)
A = np.zeros((N, N))
K = np.zeros(N)
for a, b in top_N_edges:
    A[a,b] = 1 
    K[b] += 1
A_tr = dag.transitive_reduction(A)

15460


In [ ]:
LP = dag.longest_path_matrix(A)
max_sep = int(np.max(LP)/2)
X = dag.naive_spacelike_matrix(LP, max_sep)

In [3]:
def make_or_load_dataset(filepath, X=None, D=None, A=None):
    try:
        with open(filepath, 'rb') as f:
            R = pickle.load(f)
        print 'Loaded data'
        return R
    except:
        assert X is not None
        assert D is not None
        assert A is not None
        R = dag.MDS(X, D, method='lorentzian')
        R = util.flip_time_axis(R, A)
        with open(filepath, 'wb') as f:
            pickle.dump(R, f)
        print 'Writing data'
        return R

In [4]:
filepath = './results/hep-th_1000.pkl'
#filepath = './datasets/random_dag/N1000_k10.pkl'
R = make_or_load_dataset(filepath)

Loaded data


In [28]:
def plot_embedding_2d(R, A=None, figname=None):
    N, D_max = R.shape
    colors = np.arange(N)/float(N)
    fig, ax = plt.subplots()
    ax.axes.get_yaxis().set_ticks([])
    ax.axes.get_xaxis().set_ticks([])
    ax.set_xlabel('Space')
    ax.set_ylabel('Time')
    cax = ax.scatter(R[:,1], R[:,0], 
                     s=K*2, c=colors, cmap='plasma', linewidth=0.5)
    cbar = fig.colorbar(cax, #ticks=[min(K), np.mean(K), max(K)-1],
                       orientation='horizontal')
    cbar.ax.set_xticklabels(
        ['', 'Older papers', '','','','','','','','Newer papers'])
    if A is not None:
        for i in range(N):
            for j in range(N):
                if A[i,j] == 1:
                    ax.plot([R[i,1], R[j,1]], [R[i,0], R[j,0]],
                           'k-', alpha=0.00)
    if figname:
        plt.savefig('../figures/' + figname + '.pdf')
        plt.savefig('../figures/' + figname + '.png')
        plt.savefig('../figures/' + figname + '.eps')
    plt.show()

In [17]:
def plot_embedding_3d(R, A=None):
    N, D_max = R.shape
    colors = np.arange(N)/float(N)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(R[:,2], R[:,1], R[:,0], s=K*2, c=colors, cmap='viridis')
    
    
    if A is not None:
        for i in range(N):
            for j in range(N):
                if A[i,j] == 1:
                    ax.plot([R[i,2], R[j,2]], [R[i,1], R[j,1]], [R[i,0], R[j,0]],
                           'k-', alpha=0.02)
    plt.show()

In [29]:
plot_embedding_2d(R, A_tr, 'hep-th_1000')